In [ ]:
import cache
import meta
import meta_session

import nept
import numpy as np
import pandas as pd
import scipy.stats
import scipy.signal
from scipy.ndimage import median_filter
from shapely.geometry import LineString, Point
from shapely.ops import split
import statsmodels.api as sm
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator

from utils import simple_ttest
from utils import save_ttest_results

%matplotlib inline

In [ ]:
info = meta_session.r066d2
group = "combined"

def get(key, info=None, group=None):
    if info is not None:
        return cache.load(f"ind-{info.session_id}", key)
    return cache.load(f"grp-{group}", key)

In [ ]:
swr_n_byphase = get("swr_n_byphase", group=group)
replay_n_byphase = get("replay_n_byphase", group=group)

In [ ]:
def simple_ttest(n1, p1, n2, p2):
    rng = np.random.RandomState(meta.seed + 2)
    population1 = rng.binomial(1, p1, n1)
    population2 = rng.binomial(1, p2, n2)
    t_stat, p_val, n = sm.stats.ttest_ind(population1, population2)
    return p_val

In [ ]:
def get_replay_proportions_byphase_pval(swr_n_byphase, replay_n_byphase):
    pval = {}
    for trajectory in replay_n_byphase:
        pval[trajectory] = {}
        for left, right in zip(
            meta.rest_times[:-1] + meta.run_times[:-1],
            meta.rest_times[1:] + meta.run_times[1:],
        ):
            pval[trajectory][(left, right)] = (
                simple_ttest(
                    n1=swr_n_byphase[left],
                    p1=replay_n_byphase[trajectory][left] / swr_n_byphase[left],
                    n2=swr_n_byphase[right],
                    p2=replay_n_byphase[trajectory][right] / swr_n_byphase[right],
                )
                if swr_n_byphase[left] > 0 and swr_n_byphase[right] > 0
                else 1.0
            )

    for key in ["overlapping", "exclusive"]:
        prefix = "only_" if key == "exclusive" else ""
        pval[key] = {
            task_time: simple_ttest(
                n1=swr_n_byphase[task_time],
                p1=replay_n_byphase[f"{prefix}u"][task_time] / swr_n_byphase[task_time],
                n2=swr_n_byphase[task_time],
                p2=replay_n_byphase[f"{prefix}full_shortcut"][task_time]
                / swr_n_byphase[task_time],
            )
            if swr_n_byphase[task_time] > 0
            else 1.0
            for task_time in meta.task_times
        }

    return pval

In [ ]:
pval = get_replay_proportions_byphase_pval(swr_n_byphase, replay_n_byphase)
pval

In [ ]:
def simple_ttest(n1, p1, n2, p2):
    rng = np.random.RandomState(meta.seed + 2)
    population1 = rng.binomial(1, p1, n1)
    population2 = rng.binomial(1, p2, n2)
    t_stat, p_val, n = sm.stats.ttest_ind(population1, population2)
    return p_val

def get_replay_proportions_byphase_pval(swr_n_byphase, replay_n_byphase):
    pval = {}
    for trajectory in replay_n_byphase:
        pval[trajectory] = {}
        for left, right in zip(
            meta.rest_times[:-1] + meta.run_times[:-1],
            meta.rest_times[1:] + meta.run_times[1:],
        ):
            pval[trajectory][(left, right)] = (
                simple_ttest(
                    n1=swr_n_byphase[left],
                    p1=replay_n_byphase[trajectory][left] / swr_n_byphase[left],
                    n2=swr_n_byphase[right],
                    p2=replay_n_byphase[trajectory][right] / swr_n_byphase[right],
                )
                if swr_n_byphase[left] > 0 and swr_n_byphase[right] > 0
                else 1.0
            )

    for key in ["overlapping", "exclusive"]:
        prefix = "only_" if key == "exclusive" else ""
        pval[key] = {
            task_time: scipy.stats.ranksums(replay_n_byphase[f"{prefix}u"][task_time] / swr_n_byphase[task_time], 
                                                          replay_n_byphase[f"{prefix}full_shortcut"][task_time] / swr_n_byphase[task_time])

            if swr_n_byphase[task_time] > 0
            else 1.0
            for task_time in meta.task_times
        }

    return pval

pval = get_replay_proportions_byphase_pval(swr_n_byphase, replay_n_byphase)
pval

In [ ]:
key = "overlapping"
task_time = "phase3"
prefix = "only_" if key == "exclusive" else ""
scipy.stats.ranksums(replay_n_byphase[f"{prefix}u"][task_time] / swr_n_byphase[task_time], 
                     replay_n_byphase[f"{prefix}full_shortcut"][task_time] / swr_n_byphase[task_time])

In [ ]:
replay_n_byphase[f"{prefix}full_shortcut"][task_time] / swr_n_byphase[task_time]

In [ ]:
replay_proportions_byphase = get("replay_proportions_byphase", group=group)

In [ ]:
replay_proportions_byphase

In [ ]:
############ appear/disappear bybin

In [ ]:
tc_appear_maxpeaks = get("tc_appear_maxpeaks", group=group)

In [ ]:
array = tc_appear_maxpeaks["phases12"]

In [ ]:
landmarks = np.zeros(meta.n_bins)

In [ ]:
meta.std_axvlines["u"]

In [ ]:
n = 1
landmark_idx = []
for pt in meta.std_axvlines["u"]:
    if pt == 0:
        landmark_idx.append(list(np.arange(pt, pt+n+1, 1)))
    elif 0 < pt < 100:
        landmark_idx.append(list(np.arange(pt-n, pt+n+1, 1)))
    elif pt == 100:
        landmark_idx.append(list(np.arange(pt-n-1, pt, 1)))
landmark_idx = np.array([idx for sublist in landmark_idx for idx in sublist])

In [ ]:
landmark_idx

In [ ]:
landmark_idx.size

In [ ]:
array

In [ ]:
array[landmark_idx]

In [ ]:
intersection_idx = [19, 20, 21, 79, 80, 81]
turn_idx = [39, 40, 41, 59, 60, 61]
end_idx = [0, 1, 98, 99]

In [ ]:
n=10000
p=10/19
k=0
scipy.stats.binom.cdf(k,n,p)

In [ ]:
n = np.sum(array[intersection_idx])        # Number of total bets
p = 1/meta.n_bins * landmark_idx.size  # Probability of getting "red" at the roulette
max_sbets = array.size  # Maximum number of successful bets

hh = scipy.stats.binom(n, p)

total_p = 0
for k in range(1, max_sbets + 1):  # DO NOT FORGET THAT THE LAST INDEX IS NOT USED
    total_p += hh.pmf(k)
total_p * 2 # times 2 to calculate two-sided probability

In [ ]:
1/100

In [ ]:
n = 15         # Number of total bets
p = 18./38     # Probability of getting "red" at the roulette
max_sbets = 4  # Maximum number of successful bets

hh = scipy.stats.binom(n, p)

total_p = 0
for k in range(1, max_sbets + 1):  # DO NOT FORGET THAT THE LAST INDEX IS NOT USED
    total_p += hh.pmf(k)
total_p

In [ ]:
######### tc correlation btw phases

In [ ]:
tc_correlations = get("tc_correlations", group=group)

In [ ]:
tc_correlations

In [ ]:
stats, pval = scipy.stats.ranksums(tc_correlations["phases12"], tc_correlations["phases23"])

In [ ]:
pval

In [ ]:
stats